In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt


In [7]:

# ========== Step 1: Settings ==========
device = torch.device('cpu')  # Force to use CPU
data_dir = r'C:\Users\BNC\Documents\ITC-project\plastic\data_plastic'
num_classes = 3
batch_size = 16
num_epochs = 50
learning_rate = 0.001
img_size = 64


In [8]:
print(f"Using device: {device}")
print(f"Data directory: {data_dir}")
print(f"Number of classes: {num_classes}")


Using device: cpu
Data directory: C:\Users\BNC\Documents\ITC-project\plastic\data_plastic
Number of classes: 3


In [9]:
train_path = os.path.join(data_dir, 'train')
val_path = os.path.join(data_dir, 'val')

print("Train path exists:", os.path.exists(train_path))
print("Val path exists:", os.path.exists(val_path))


Train path exists: True
Val path exists: True


In [10]:
# ========== Step 2: Data Preparation ==========
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# ========== Step 3: Model Definition ==========


In [11]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)


In [12]:
# ========== Step 4: Loss and Optimizer ==========
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [2]:

# ========== Step 1: Settings ==========
device = torch.device('cpu')  # Use CPU
data_dir = r'C:\Users\BNC\Documents\ITC-project\plastic\data_plastic'  # <-- update if needed
batch_size = 16
num_epochs = 50
learning_rate = 0.001
img_size = 64


In [3]:

# ========== Step 2: Data Preparation ==========
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)

# Auto-detect number of classes from folder names
num_classes = len(train_dataset.classes)
print("Detected Classes:", train_dataset.classes)
print("Total Classes:", num_classes)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


Detected Classes: ['aluminum_food_cans', 'foot', 'glass', 'paper', 'plastic']
Total Classes: 5


In [4]:

# ========== Step 3: Model Definition ==========
class SmallCNN(nn.Module):
    def __init__(self, num_classes):
        super(SmallCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * (img_size // 8) * (img_size // 8), 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SmallCNN(num_classes).to(device)


In [5]:

# ========== Step 4: Loss and Optimizer ==========
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [6]:

# ========== Step 5: Training and Validation ==========
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {epoch_loss:.4f} | Train Accuracy: {epoch_accuracy:.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_epoch_loss = val_loss / len(val_loader.dataset)
    val_epoch_accuracy = val_correct / val_total
    val_losses.append(val_epoch_loss)
    val_accuracies.append(val_epoch_accuracy)

    print(f"Validation Loss: {val_epoch_loss:.4f} | Validation Accuracy: {val_epoch_accuracy:.4f}")

# ========== Step 6: Save Model ==========
torch.save(model.state_dict(), "small_model_cnn.pth")
print("✅ Model saved to small_cnn_model.pth")


Epoch [1/50] | Train Loss: 1.0930 | Train Accuracy: 0.5855
Validation Loss: 0.8733 | Validation Accuracy: 0.6706
Epoch [2/50] | Train Loss: 0.8272 | Train Accuracy: 0.6961
Validation Loss: 0.6519 | Validation Accuracy: 0.7570
Epoch [3/50] | Train Loss: 0.6445 | Train Accuracy: 0.7684
Validation Loss: 0.4607 | Validation Accuracy: 0.8290
Epoch [4/50] | Train Loss: 0.4843 | Train Accuracy: 0.8278
Validation Loss: 0.3688 | Validation Accuracy: 0.8800
Epoch [5/50] | Train Loss: 0.3589 | Train Accuracy: 0.8761
Validation Loss: 0.2410 | Validation Accuracy: 0.9215
Epoch [6/50] | Train Loss: 0.2530 | Train Accuracy: 0.9149
Validation Loss: 0.1612 | Validation Accuracy: 0.9513
Epoch [7/50] | Train Loss: 0.1811 | Train Accuracy: 0.9399
Validation Loss: 0.1301 | Validation Accuracy: 0.9592
Epoch [8/50] | Train Loss: 0.1442 | Train Accuracy: 0.9507
Validation Loss: 0.1100 | Validation Accuracy: 0.9661
Epoch [9/50] | Train Loss: 0.1033 | Train Accuracy: 0.9659
Validation Loss: 0.0759 | Validation 

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import cv2

# ========== Config ==========
MODEL_PATH = r"C:\Users\BNC\Documents\ITC-project\plastic\small_model_cnn.pth"
NUM_CLASSES = 5  # <-- Must match the trained model
IMAGE_SIZE = 64
CLASS_NAMES = [f'class_{i}' for i in range(NUM_CLASSES)]  # Placeholder names

# ========== Model Definition ==========
class TrainedCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(TrainedCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),  # (B,3,64,64) -> (B,16,64,64)
            nn.ReLU(),
            nn.MaxPool2d(2),                # -> (B,16,32,32)

            nn.Conv2d(16, 32, 3, padding=1),  # -> (B,32,32,32)
            nn.ReLU(),
            nn.MaxPool2d(2),                 # -> (B,32,16,16)

            nn.Conv2d(32, 64, 3, padding=1),  # This layer caused your error (features.6)
            nn.ReLU(),
            nn.MaxPool2d(2),                 # -> (B,64,8,8)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),                  # -> (B,64*8*8 = 4096)
            nn.Linear(64 * 8 * 8, 128),    # (4096 -> 128)
            nn.ReLU(),
            nn.Linear(128, num_classes)    # (128 -> num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# ========== Load Model ==========
model = TrainedCNN(num_classes=NUM_CLASSES)
model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu')))
model.eval()

# ========== Define Transform ==========
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

# ========== Real-time Camera ==========
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to grab frame")
        break

    # Resize and transform image
    img = cv2.resize(frame, (IMAGE_SIZE, IMAGE_SIZE))
    input_tensor = transform(img).unsqueeze(0)

    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)
        label = CLASS_NAMES[predicted.item()]

    # Draw prediction
    cv2.putText(frame, f"Prediction: {label}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Real-time Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 

In [ ]:
# ========== Step 6: Plotting Results ==========
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Accuracy Curve')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.savefig('training_results.png')
plt.show()
# ========== Step 7: Save the Model ==========
model_save_path = 'resnet18_plastic_classifier.pth'
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')


In [1]:
!pip install torch torchvision matplotlib

  Using cached torch-2.7.1-cp310-cp310-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.22.1-cp310-cp310-win_amd64.whl.metadata (6.1 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached contourpy-1.3.2-cp310-cp310-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp310-cp310-win_amd64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
Using cached torch-2.7.1-cp310-cp310-win_amd64.whl (216.1 MB)
Using cached torchvision-0.22.1-cp310-cp310-win_amd64.whl (1.7 MB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ------- -----------------------------

In [18]:
pip install opencv-python


  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.0 MB ? eta -:--:--
   - -------------------------------------- 1.0/39.0 MB 3.1 MB/s eta 0:00:13
   - -------------------------------------- 1.8/39.0 MB 3.6 MB/s eta 0:00:11
   -- ------------------------------------- 2.4/39.0 MB 3.2 MB/s eta 0:00:12
   --- ------------------------------------ 3.1/39.0 MB 3.4 MB/s eta 0:00:11
   ---- ----------------------------------- 3.9/39.0 MB 3.5 MB/s eta 0:00:11
   ---- ----------------------------------- 4.7/39.0 MB 3.5 MB/s eta 0:00:10
   ----- ---------------------------------- 5.2/39.0 MB 3.4 MB/s eta 0:00:11
   ----- ---------------------------------- 5.8/39.0 MB 3.2 MB/s eta 0:00:11
   ------ --------------